# Lecture 5 - Unit testing

## 📕 Today's Agenda
---
 * [Unittest module](#Unittest-module)
 * [Simple unit test](#Simple-unit-test)
 * [Executing tests](#Executing-tests)
 * [Test methods](#Test-methods)
 * [Setup and teardown](#Setup-and-teardown)
 * [Mocking](#Mocking)
 * [Patching](#Patching)
 * [Skipping tests and errors](#Skipping-tests-and-errors)

## 🧪 Theory
---

### Unittest module

There are many modules for testing Python code, one on we will focus on is `unittest`. Other well known module for testing is `pytest`.
Unittest is a entire framework for testing your Py code, you can make tests, test suites and run tests with built-in runner.


### Simple unit test

Let's build a simple test for a simple function.

In [2]:
from math import pi
import unittest

def circle_area(radius):
    if not isinstance(radius, float) or not isinstance(radius, int):
        raise TypeError('Radius must be integer of float.')
    if radius <= 0:
        raise ValueError('Radius must be greater then zero.')
    return pi * radius ** 2

class TestMyFunctions(unittest.TestCase):
    def test_circle_area(self):
        with self.assertRaises(TypeError):
            circle_area(None)
        with self.assertRaises(ValueError):
            circle_area(-1)
        self.assertEqual(circle_area(1), 0.7853981634)

First steps to consider when creating tests:
1. Create a class that extends `unittest.TestCase'. Name this class prefixed with 'Test'.
2. Create functions in this class for testing functions. Each test function name must be prefixed with 'test_'.
3. Write test code in test functions.

NOTE: Prefixing classes and functions names will help test framework to auto-discover test.

### Executing tests

To run a python file with tests, place this snippet at the bottom.
```python
if __name__ == '__main__':
    unittest.main()
```

### Test methods
The `TestCase` class provides several assert methods to check for and report failures. The following table lists the most
commonly used methods (see the tables below for more assert methods):

| Method | Checks that |
| :-: | :-: |
| assertEqual(a, b) | a == b |
| assertNotEqual(a, b) | a != b |
| assertTrue(x) | x is True |
| assertFalse(x) | x is False |
| assertIs(a, b) | a is b |
| assertIsNot(a, b) | a is not b |
| assertIsNone(x) | x is None |
| assertIsNotNone(x) | x is not None |
| assertIn(a, b) | a in b |
| assertNotIn(a, b) | a not in b |
| assertIsInstance(a, b) | a is instance of b |
| assertNotIsInstance(a, b) | a is not instance of b |
| assertRaises(x)* | check if x is raised |

\* Used only with context manager

### Setup and teardown

The `TestCase` class exposes `setUp` and `tearDown` methods for housekeeping purposes. They are not implemented so you
have to do it in your test case.

In the *setUp* method you can place code to be run before each test executes and in *tearDown* method you can place code to be
that will be executed after each test.

In [3]:
class TestMx(unittest.TestCase):

    def setUp(self):
        print('Executing new test...')

    def test_one(self):
        self.assertEquals(1, 1)

    def test_two(self):
        self.assertEqual(1, 1)

    def tearDown(self):
        print('DONE')

### Mocking

From testing perspective mocking is the act of crating dummy objects who emulates an object needed by a function.
If you have a functions which extracts information from a HTTP response object, and you want to test that function, you will need
a HTTP response object in different states. Because you want to test your function for all possible responses.
So you will need a good HTTP response object and may bad responses. You don't want to stress the production server with your
requests or to make tricks so you will obtain the needed response type.

In [4]:
class HttpError(Exception):
    pass

class HttpResponse:
    def __init__(self):
        self.status = None

    def json(self):
        return {}

def store_car_info(http_response):
    """This function stores car info data received from http response to disk."""
    if not isinstance(http_response, HttpResponse):
        raise TypeError('HttpResponse expected, got other type.')
    if http_response.status == 200:
        content = http_response.json()
        car_id = content.get('car_id')
        if car_id:
            print(f'Storing data for {car_id} ...')
            # suppose you store data
            print('Done.')
        else:
            raise ValueError('Invalid http content.')
    else:
        raise HttpError(f'Http error, status: {http_response.status}')

So, we got `store_car_function` and has to be tested. **Let's define tests to perform!**

Import `Mock` class and make a mock object that emulates a HttpResponse object.

In [5]:
from unittest.mock import Mock

mock_response = Mock(spec_set=HttpResponse)
print(isinstance(mock_response, HttpResponse))

True


The *Mock* class allow us to define any attribute or method. You can specify values for attributes,
return values for methods or errors to raise.

**Set value for attributes**

In [6]:
mock_response2 = Mock()
mock_response2.status = 200
print(mock_response2.status)

200


**Set return value for methods**

In [7]:
mock_response2.json.return_value = {'car_id': 2}
print(mock_response2.json())

{'car_id': 2}


**Raise errors**

In [8]:
mock_response2.json.side_effect = TypeError('Error raised by mock')
try:
    mock_response2.json()
except TypeError as err:
    print(err)

Error raised by mock


### Patching

Well, to test a function that has external dependencies is quite easy, just use Mocking, but how you test a function that has
has dependencies on external libraries, like datetime for example. How you test a function that internally grabs the datetime
and makes some decisions.

Like a function for greeting:

In [9]:
from datetime import datetime

def greet():
    now = datetime.now()
    if 5 < now.hour <= 10:
        return 'Good morning!'
    elif 10 < now.hour <= 19:
        return 'Good afternoon!'
    else:
        return 'Good evening!'

To test the `greet` function, we have to change the return value for `datetime.now()` temporally. We can do this by using `patch` function.
The best way of using it is in a context manager.

In [10]:
from unittest.mock import patch

class TestMyFunctions(unittest.TestCase):
    def test_greet(self):
        # test for evening
        with patch('.datetime') as mock:
            mock.now.return_value = datetime(2020, 1, 1, 20, 20)
            self.assertEqual('Good evening!', greet())

Let's break it down, so with `patch('.datetime') as mock` we create a mock for built-in module datetime that was imported in our script
and set a return value for `now()` method of datetime module. The new value is used just in the actual context.

### Skipping tests and errors
Unittest supports skipping individual test methods and even whole classes of tests. In addition, it supports marking a
test as an “expected failure,” a test that is broken and will fail, but shouldn’t be counted as a failure on a TestResult.

Skipping a test is simply a matter of using the `skip()` **decorator** or one of its conditional variants.
Unittest supports skipping individual test methods and even whole classes of tests. In addition, it supports marking a
test as an “expected failure,” a test that is broken and will fail, but shouldn’t be counted as a failure on a TestResult.

In [ ]:
import sys
class MyTestCase(unittest.TestCase):

    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(sys.version < '3',
                     "not supported in this Python version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

    def test_maybe_skipped(self):
        # test code that depends on the external resource
        pass

To skip a class of tests just decorate the class with `@unittest.skip`, and provide message as argument.                                                                                                                                                                                 Skipping a test is simply a matter of using the skip() decorator or one of its conditional variants, calling TestCase.skipTest() within a setUp() or test method, or raising SkipTest directly.

When you expect a test to fail, just decorate test function with `@unittest.expectFailure` and that test will be marked as
*expected failure* in test report. If a test that is expected to fail, succeeds it will be marked as *unexpected success*.

## 👩‍💻 Practice
---
1. Define test that have to be run for `store_car_info`.
2. Write unit tests for `store_car_info` function. Use Mock.

## 🏠 Homework
---